In [ ]:
import os
from itertools import product
from pandas import concat, DataFrame, set_option
from seaborn import set_theme
os.chdir("../")
from src.utils import _plot_confusion_matrix
from src.data import Format
from src.models import Pipeline

In [ ]:
set_option('display.max_columns', None)
set_theme()

# Prediction Global Report

In [ ]:
""" initialize variables """
_list_datasets_T = [
                    #["mrda", 50],
                    #["swda", 50],
                    ["dyda_da", 5],
                    ["dyda_e", 5],
                    #["meld_s", 5],
                    #["meld_e", 5]
                    ]

_list_encoder = ["bert-base-cased", "xlnet-base-cased"]

_list_data_format_decoder = [["MLP", "concatenate"], ["GRU", "separate"]]

_list_inputs = list(product(_list_datasets_T, _list_encoder, _list_data_format_decoder))

_possible_pipelines = {"inputs": _list_inputs, "outputs": []}

df_global_report = DataFrame()

In [4]:
for _input in _possible_pipelines["inputs"]:
    """compile the model"""
    _pipeline = Pipeline(_input[0][0], _input[0][1], _input[1], _input[2][0], _input[2][1])
    _pipeline.summarize()

    """store outputs"""
    _pipeline.df_report.to_csv(
                 f"./data/outputs_data/df_{_input[0][0]}_{_input[1].split('-')[0]}_{_input[2][0]}.csv",
                 index=False,
                 sep="|",
                 encoding='utf-8')
    df_global_report = concat([df_global_report, _pipeline.df_report]).reset_index(drop=True)
    _possible_pipelines["outputs"].append(_pipeline.confusion_matrix)

    """plot and save the confusion matrix"""
    _list_labels = Format(_input[0][0], _input[0][1], _input[2][1]).get_distincts_labels()
    _plot_confusion_matrix(_pipeline.confusion_matrix, _input, _list_labels)

In [ ]:
df_global_report